<img src='steane.svg'></img>
$$H_X = H_Z = \begin{bmatrix} 0 & 0 & 0 & 1 & 1 & 1 & 1 \\ 
                              0 & 1 & 1 & 0 & 0 & 1 & 1 \\ 
                              1 & 0 & 1 & 0 & 1 & 0 & 1 \end{bmatrix}$$
$$|0\rangle_L = \frac{1}{2\sqrt{2}}\biggl(|0000000\rangle + |1010101\rangle + |0110011\rangle + |1100110\rangle +
                                          |0001111\rangle + |1011010\rangle + |0111100\rangle + |1101001\rangle \biggr),$$
$$|1\rangle_L = \frac{1}{2\sqrt{2}}\biggl(|1111111\rangle + |0101010\rangle + |1001100\rangle + |0011001\rangle +
                                          |1110000\rangle + |0100101\rangle + |1000011\rangle + |0010110\rangle \biggr).$$
$$X_L|0\rangle_L = |1\rangle_L$$

In [2]:
from stim import Tableau, Circuit, PauliString, TableauSimulator
import pymatching 
from scipy import sparse
import scipy.linalg as lin
import numpy as np

In [3]:
def get_stabilizers_from_tableau(s: TableauSimulator):
    return s.current_inverse_tableau().inverse().to_stabilizers()[:-1]

In [4]:
steane_generator_group = [
    PauliString('___ZZZZ'), # Z Hamming matrix # S_{Z, 1}
    PauliString('_ZZ__ZZ'),                    # S_{Z, 2}
    PauliString('Z_Z_Z_Z'),                    # S_{Z, 3}
    PauliString('___XXXX'), # X Hamming matrix # S_{X, 1}
    PauliString('_XX__XX'),                    # S_{X, 2}
    PauliString('X_X_X_X'),                    # S_{X, 3}
    PauliString('ZZZZZZZ') # Logical
]
X_L = PauliString('XXXXXXX')
Z_L = PauliString('ZZZZZZZ')

s = TableauSimulator()
s.set_state_from_stabilizers(steane_generator_group)

In [5]:
logical_0 = np.zeros((2**7,)) # Sanity check
np.put(logical_0, [int(0b0000000), int(0b1010101), int(0b1100110), int(0b0110011), 
                   int(0b1111000), int(0b0101101), int(0b0011110), int(0b1001011)], 1/(2*np.sqrt(2)))
np.allclose(s.state_vector(), logical_0)

True

$H_X$, $H_Z$ are both the $[7,4, 3]$ Hamming parity check matrices. 

$$H_X \cdot H_Z^T \pmod{2} = 0$$ 

$$ \bold{H}= \left[\begin{array}{c|c} H_X & 0 \\ 0 & H_Z \end{array}\right] = 
                    \left[ \begin{array}{ccccccc|ccccccc} 
                              0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 
                              0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 
                              1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
                              0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 \\ 
                              0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 \\ 
                              0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1
                              \end{array} \right].$$ 

We can also demonstrate that the stabilizers commute with the binary symplectic product of $\bold{H}$:
$$\bold{H} \odot \bold{H} = 0,$$
is the binary symplectic product $A\odot B \equiv A \Omega B, \  \Omega = \begin{bmatrix} 0 & I_n \\ I_n & 0 \end{bmatrix}  $ is the binary symplectic matrix ($n=7$ in our case)

In [6]:
H_ZX = np.matrix([[0, 0, 0, 1, 1, 1, 1], 
                  [0, 1, 1, 0, 0, 1, 1], 
                  [1, 0, 1, 0, 1, 0, 1]], )
H_X = np.concatenate([H_ZX, np.zeros((3,7))], axis=0)
H_Z = np.concatenate([np.zeros((3,7)), H_ZX], axis=0)
H = lin.block_diag(H_ZX, H_ZX)
symp_bin_mat = np.kron(np.eye(2)[::-1], np.eye(7)) # [ 0 & I_7 // I_7 & 0]

# 0 if 0 matrix as described
np.count_nonzero(np.mod(np.dot(H_X, H_Z.T), 2).sum())

0

In [8]:
c = Tableau(7).from_stabilizers(steane_generator_group).to_circuit()
c.append_from_stim_program_text(
'''
    # Bit-flip parity check
    CX 3 7 4 7 5 7 6 7
    CX 1 8 2 8 5 8 6 8
    CX 0 9 2 9 4 9 6 9

    # Phase-flip parity check
    H 10 11 12
    CX 10 3 10 4 10 5 10 6
    CX 11 1 11 2 11 5 11 6
    CX 12 0 12 2 12 4 12 6
    H 10 11 12

    # Measure parity
    M 7 8 9 10 11 12                      
''')
c.to_tableau(ignore_measurement=True).to_stabilizers()

[stim.PauliString("+___ZZZZ______"),
 stim.PauliString("+_ZZ__ZZ______"),
 stim.PauliString("+Z_Z_Z_Z______"),
 stim.PauliString("+___XXXX______"),
 stim.PauliString("+_XX__XX______"),
 stim.PauliString("+X_X_X_X______"),
 stim.PauliString("+ZZZZZZZ______"),
 stim.PauliString("+___ZZZZZ_____"),
 stim.PauliString("+_ZZ__ZZ_Z____"),
 stim.PauliString("+Z_Z_Z_Z__Z___"),
 stim.PauliString("+___XXXX___Z__"),
 stim.PauliString("+_XX__XX____Z_"),
 stim.PauliString("+X_X_X_X_____Z")]